In [2]:
print(os.getcwd())


J:\LEGAL\share1\IP_forecasting_test_env


In [1]:
print('hello world')
import pandas as pd
import numpy as np
import shutil
import itertools
import calendar
from glob import glob
# import pandas_profiling
from datetime import timedelta, datetime
# pd.options.display.float_format = '${:,.0f}'.format
pd.options.display.float_format = '{:,.0f}'.format
import os
print(os.getcwd())
os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
print(os.getcwd())
# pd.show_versions()
print(pd.__version__)
np.__version__

hello world
C:\Users\hansoc10\Desktop\Programming\Jupyter
J:\LEGAL\share1\IP_forecasting_test_env
0.24.2


'1.16.3'

In [ ]:
'''
Make dataframe with the current master forecast REV #1 OBSOLETE
'''
current_forecast_MDT = pd.read_excel('current_forecast\MDT_current_forecast_rollup.xlsx')


'''
create list of new forecasts
'''
files_to_process = sorted(glob('*CAH_forecast*.xlsx'))
print(files_to_process)

# '''
# update the files to process to add the fiscal quarters and other info
# '''
# for file in files_to_process:
#     df = pd.read_excel(file)
#     df['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR')
#     # print(df.head())
#     df['month'] = df.task_date.dt.month
#     df['year'] = df.task_date.dt.year
#     df['day'] = df.task_date.dt.day
#     df['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR').quarter
#     df.loc[((df.task_date.dt.month >= 5) | ((df.task_date.dt.month == 4) & (df['fiscal_qtr'] == 1))), 'fiscal_yr'] = df.task_date.dt.year + 1
#     df.loc[((df.task_date.dt.month <= 4) & (df['fiscal_qtr'] != 1)), 'fiscal_yr'] = df.task_date.dt.year
#     df.to_excel(file, index=False)



'''
for all the forecasts in the list of files to process
create a single dataframe of the new forecasts by concatenating them into a single dataframe
give them a timestamp so we know when it occurred as this will allows us to see all historical forecasts
create or update the forecast status to current
'''
new_forecasts_to_process = pd.concat((pd.read_excel(file) for file in files_to_process), ignore_index=True, sort=False)
new_forecasts_to_process['timestamp'] = pd.Timestamp.today()
new_forecasts_to_process['frcst_status'] = 'current'
# print(new_forecasts_to_process.columns)


'''
create a list of features in the new, aka, current, forecasts that can distiguish these new or updated forecasts 
from forecasts that are current forecasts in the current master forecast.
These lists will be used to change the status of the previous current forecast to obsolete
because these new forecasts are to be considered the current forecast.
'''
frmnme = new_forecasts_to_process['firm_name'].unique()
mtrnum = new_forecasts_to_process['matter_num'].unique()
# print(frmnme)
# print(mtrnum)

'''
The attorneys and firms are asked to fill out the task_date column for their forecasts
using that date, this code updates a number of series columns that 
make it easy to slice and dice the forecast.
Importantly, this does it so that the MDT fiscal year is used exactly.
MDT is a 5253 type fiscal year and the way the programming works is to match the last friday of the quarter
that there is a step to use a day offset of 1 to make sure we get it right for the fiscal year to the day
and this last step is necessary because the fiscal offsets for when done for the quarter doesn't get the fiscal year correct
'''

new_forecasts_to_process['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in new_forecasts_to_process.task_date], freq='Q-APR')
new_forecasts_to_process['month'] = new_forecasts_to_process.task_date.dt.month
new_forecasts_to_process['year'] = new_forecasts_to_process.task_date.dt.year
new_forecasts_to_process['day'] = new_forecasts_to_process.task_date.dt.day
new_forecasts_to_process['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in new_forecasts_to_process.task_date], freq='Q-APR').quarter

new_forecasts_to_process.loc[((new_forecasts_to_process.task_date.dt.month >= 5) | ((new_forecasts_to_process.task_date.dt.month == 4) & (new_forecasts_to_process['fiscal_qtr'] == 1))), 'fiscal_yr'] = new_forecasts_to_process.task_date.dt.year + 1
new_forecasts_to_process.loc[((new_forecasts_to_process.task_date.dt.month <= 4) & (new_forecasts_to_process['fiscal_qtr'] != 1)), 'fiscal_yr'] = new_forecasts_to_process.task_date.dt.year



'''
see if any of the new forecasts to process for a given firm and a resulting matter
are already in the master database as a current forecast
if so, change the current forecast frcst status in the master database
from "current" to "obsolete".
then late in the program the new_forecasts_to_process will be added as current forecasts
to do this identify the rows by firm name and matter number and put in variable rows
for those rows change forecast status to obsolete to prepare for the new forecasts
'''
# print(len(frmnme))
for num in range(len(frmnme)):
    rows = (current_forecast_MDT['firm_name'] == frmnme[num]) & (current_forecast_MDT['matter_num'] == mtrnum[num]) & (current_forecast_MDT['frcst_status'] == 'current')
    current_forecast_MDT.loc[rows, ['frcst_status']] = ['obsolete']

'''
add new forecasts to the current forecast creating an updated master forecast
'''
updated_forecast = pd.concat([current_forecast_MDT, new_forecasts_to_process], ignore_index=True, sort=False)

'''
write the newly created current forecast back to excel.
'''
updated_forecast.to_excel('current_forecast/MDT_current_forecast_rollup.xlsx', index=False)

'''
adding timestamp to files_to_process
'''
timestmp_prefix = (str(pd.Timestamp.today().year)
                   + str(pd.Timestamp.today().month)
                   + str(pd.Timestamp.today().day)
                   + str(pd.Timestamp.today().hour) 
                   + str(pd.Timestamp.today().minute))

'''
i read the following list comprehension as taking file f and formatting the ouput name 
to be the prefix followed by the original file name
for all files in the directory meeting the filename criteria in the glob function above
'''
[os.rename(f, "{}{}".format(timestmp_prefix, f)) for f in files_to_process]

'''
move the forecasts that were the new to be forecasts 
that are now processed and have a timestamp prefix added to their name 
to a processed file folder "submitted_forecast_processed"
so we don't process the forecasts twice
'''
files_to_move = sorted(glob('*CAH_forecast*.xlsx'))
# print(files_to_process)

# THIS COMMENT SECTION NEEDS TO BE TURNED ON WHEN THE PROGRAM IS IN USE
# destination = "submitted_forecast_processed/"
# for files in files_to_move:
#     if files.endswith(".xlsx"):
#         shutil.move(files, destination)



# test_df = new_forecasts_to_process[['task','firm_name', 'matter_num', 'timestamp', 'frcst_status']]
test_df = current_forecast_MDT[['task','firm_name', 'matter_num', 'timestamp', 'frcst_status']]

# print(test_df)
# updated_forecast
new_forecasts_to_process.head()

In [ ]:
'''
read in excel current master forecast into dataframe REV #2
'''
current_forecast_MDT = pd.read_excel('current_forecast\MDT_current_forecast_rollup.xlsx')
# print(current_forecast_MDT.head())

'''
create list of new forecasts
'''
files_to_process = sorted(glob('*CAH_forecast*.xlsx'))
print(files_to_process)

'''
update the files to process to add the fiscal quarter info and save back to same name
'''
for file in files_to_process:
    df = pd.read_excel(file)
    df['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR')
    # print(df.head())
    df['month'] = df.task_date.dt.month
    df['year'] = df.task_date.dt.year
    df['day'] = df.task_date.dt.day
    df['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR').quarter
    df.loc[((df.task_date.dt.month >= 5) | ((df.task_date.dt.month == 4) & (df['fiscal_qtr'] == 1))), 'fiscal_yr'] = df.task_date.dt.year + 1
    df.loc[((df.task_date.dt.month <= 4) & (df['fiscal_qtr'] != 1)), 'fiscal_yr'] = df.task_date.dt.year
    df.to_excel(file, index=False)



'''
for all the forecasts in the list of files to process
create a single dataframe of the new forecasts by concatenating them into a single dataframe
give them a timestamp so we know when it occurred as this will allows us to see all historical forecasts
create or update the forecast status to current
'''
new_forecasts_to_process = pd.concat((pd.read_excel(file) for file in files_to_process), ignore_index=True, sort=False)
new_forecasts_to_process['timestamp'] = pd.Timestamp.today()
new_forecasts_to_process['frcst_status'] = 'current'
#new_forecasts_to_process['firm_name'] = [str(i).lower() for i in new_forecasts_to_process['firm_name']]
new_forecasts_to_process['firm_name'] = [str(i).casefold() for i in new_forecasts_to_process['firm_name']]
new_forecasts_to_process['matter_name'] = [str(i).casefold() for i in new_forecasts_to_process['matter_name']]
# print(new_forecasts_to_process.columns)


'''
create a list of features in the new, aka, current, forecasts that can distiguish these new or updated forecasts 
from forecasts that are current forecasts in the current master forecast.
These lists will be used to change the status of the previous current forecast to obsolete
because these new forecasts are to be considered the current forecast.
'''
# create a row identifier by zipping two columns/series together
zipped_clms = itertools.zip_longest(new_forecasts_to_process['firm_name'], new_forecasts_to_process['matter_num'], fillvalue='whats_up')

# make a list of tuples of the unique row identifiers to identify rows in the master forecast that will become obsolete
new_forecast_identifier = []
for item in zipped_clms:
    if item not in new_forecast_identifier:
        new_forecast_identifier.append(item)

# use list comprehension to make a list of the unique firm name matter number combinations
# and use the list to set frcst_status
frmnme = [x[0] for x in new_forecast_identifier] # firm name
mtrnum = [x[1] for x in new_forecast_identifier] # matter number
print(frmnme)
print(mtrnum)




# frmnme = new_forecasts_to_process['firm_name'].unique()
# mtrnum = new_forecasts_to_process['matter_num'].unique()
# print(frmnme)
# print(mtrnum)

# '''
# The attorneys and firms are asked to fill out the task_date column for their forecasts
# using that date, this code updates a number of series columns that 
# make it easy to slice and dice the forecast.
# Importantly, this does it so that the MDT fiscal year is used exactly.
# MDT is a 5253 type fiscal year and the way the programming works is to match the last friday of the quarter
# that there is a step to use a day offset of 1 to make sure we get it right for the fiscal year to the day
# and this last step is necessary because the fiscal offsets for when done for the quarter doesn't get the fiscal year correct
# '''

# new_forecasts_to_process['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in new_forecasts_to_process.task_date], freq='Q-APR')
# new_forecasts_to_process['month'] = new_forecasts_to_process.task_date.dt.month
# new_forecasts_to_process['year'] = new_forecasts_to_process.task_date.dt.year
# new_forecasts_to_process['day'] = new_forecasts_to_process.task_date.dt.day
# new_forecasts_to_process['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in new_forecasts_to_process.task_date], freq='Q-APR').quarter

# new_forecasts_to_process.loc[((new_forecasts_to_process.task_date.dt.month >= 5) | ((new_forecasts_to_process.task_date.dt.month == 4) & (new_forecasts_to_process['fiscal_qtr'] == 1))), 'fiscal_yr'] = new_forecasts_to_process.task_date.dt.year + 1
# new_forecasts_to_process.loc[((new_forecasts_to_process.task_date.dt.month <= 4) & (new_forecasts_to_process['fiscal_qtr'] != 1)), 'fiscal_yr'] = new_forecasts_to_process.task_date.dt.year



'''
see if any of the new forecasts to process for a given firm and a resulting matter
are already in the master database as a current forecast
if so, change the current forecast frcst status in the master database
from "current" to "obsolete".
then later in the program the new_forecasts_to_process will be added as current forecasts
to do this identify the rows by firm name and matter number and put in variable rows
for those rows change forecast status to obsolete to prepare for the new forecasts
'''
print(len(frmnme))
for num in range(len(frmnme)):
    rows = (current_forecast_MDT['firm_name'] == frmnme[num]) & (current_forecast_MDT['matter_num'] == mtrnum[num]) & (current_forecast_MDT['frcst_status'] == 'current')
    current_forecast_MDT.loc[rows, ['frcst_status']] = ['obsolete']
print(rows)
print('hmm')
print(current_forecast_MDT.head())
    
'''
add new forecasts to the current forecast creating an updated master forecast
'''
updated_forecast = pd.concat([current_forecast_MDT, new_forecasts_to_process], ignore_index=True, sort=False)

'''
write the newly created current forecast back to excel.
'''
updated_forecast.to_excel('current_forecast/MDT_current_forecast_rollup.xlsx', index=False)

'''
adding timestamp to files_to_process that are now processed
'''
timestmp_prefix = (str(pd.Timestamp.today().year)
                   + str(pd.Timestamp.today().month)
                   + str(pd.Timestamp.today().day)
                   + str(pd.Timestamp.today().hour) 
                   + str(pd.Timestamp.today().minute))

'''
i read the following list comprehension as taking file f and formatting the ouput name 
to be the prefix followed by the original file name
for all files in the directory meeting the filename criteria in the glob function above
'''
[os.rename(f, "{}{}".format(timestmp_prefix, f)) for f in files_to_process]

'''
move the forecasts that were the new to be forecasts 
that are now processed and have a timestamp prefix added to their name 
to a processed file folder "submitted_forecast_processed"
so we don't process the forecasts twice
'''
files_to_move = sorted(glob('*CAH_forecast*.xlsx'))
# files_to_move = file_to_process

destination = "submitted_forecast_processed/"
for files in files_to_move:
    if files.endswith(".xlsx"):
        shutil.move(files, destination)



# test_df = new_forecasts_to_process[['task','firm_name', 'matter_num', 'timestamp', 'frcst_status']]
test_df = current_forecast_MDT[['task','firm_name', 'matter_num', 'timestamp', 'frcst_status']]
new_forecasts_to_process.head()

In [ ]:
'''
Just process one or more spreadsheets to add the fiscal information and year as we work with the forecast.
This does not create an updated forecast
'''

'''
create list of new forecasts
'''
files_to_process = sorted(glob('*CAH_forecast*.xlsx'))
print(files_to_process)

'''
update the files to process to add the fiscal quarter info and save back to same name
'''
for file in files_to_process:
    df = pd.read_excel(file)
    df['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR')
    # print(df.head())
    df['month'] = df.task_date.dt.month
    df['year'] = df.task_date.dt.year
    df['day'] = df.task_date.dt.day
    df['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR').quarter
    df.loc[((df.task_date.dt.month >= 5) | ((df.task_date.dt.month == 4) & (df['fiscal_qtr'] == 1))), 'fiscal_yr'] = df.task_date.dt.year + 1
    df.loc[((df.task_date.dt.month <= 4) & (df['fiscal_qtr'] != 1)), 'fiscal_yr'] = df.task_date.dt.year
    df.to_excel(file, index=False)

    '''
adding timestamp to files_to_process that are now processed
'''
timestmp_prefix = ('REVISED'
                   + str(pd.Timestamp.today().year)
                   + str(pd.Timestamp.today().month)
                   + str(pd.Timestamp.today().day)
                   + str(pd.Timestamp.today().hour) 
                   + str(pd.Timestamp.today().minute))

'''
i read the following list comprehension as taking file f and formatting the ouput name 
to be the prefix followed by the original file name
for all files in the directory meeting the filename criteria in the glob function above
'''
[os.rename(f, "{}{}".format(timestmp_prefix, f)) for f in files_to_process]

'''
move the forecasts that were the new to be forecasts 
that are now processed and have a timestamp prefix added to their name 
to a processed file folder "submitted_forecast_processed"
so we don't process the forecasts twice
'''
files_to_move = sorted(glob('*CAH_forecast*.xlsx'))
# files_to_move = file_to_process

destination = "forecasts_under_construction/"
for files in files_to_move:
    if files.endswith(".xlsx"):
        shutil.move(files, destination)


In [5]:
'''
Just noodling finishing touches for resulting data
This does not create an updated forecast fixed 12/4/2019, and 2/4/2020, and 7/1/2020
making some changes on 8/18/2020 regarding months 
see https://stackoverflow.com/questions/37625334/python-pandas-convert-month-int-to-month-name
https://www.geeksforgeeks.org/python-pandas-apply/
https://towardsdatascience.com/mastering-dates-and-timestamps-in-pandas-and-python-in-general-5b8c6edcc50c


'''

'''
create list of new forecasts
'''
files_to_process = sorted(glob('*CAH_KEY_STRATEGY_DOC*.xlsx'))
print(files_to_process)

'''
update the files to process to add the fiscal quarter info and save back to same name
    with pd.ExcelWriter('output.xlsx', datetime_format='YYYY-MM-DD HH:MM:SS') as writer:
    
From wikipedia: 52–53-week fiscal year
The 52–53-week fiscal year is a variation on the 4–4–5 calendar. It is used by companies that desire that 
their fiscal year always end on the same day of the week. Any day of the week may be used, and Saturday and Sunday 
are common because the business may more easily be closed for counting inventory and 
other end-of-year accounting activities.

The end of the fiscal year moves one day earlier on the calendar each year 
(or two days when there is an intervening leap day) until it would otherwise reach the date 
four days before the end of the month (August 27 in this case) or earlier. 
At that point the first Saturday in the following month (September 3 or earlier in this case) 
becomes the date closest to the end of August and it resets to that date and 
the fiscal year has 53 weeks instead of 52. In this example the fiscal years ending in 2016, 2022, and 2028 
have 53 weeks.

The 52–53 week method is permitted by generally accepted accounting principles in the United States, 
by US Internal Revenue Code Regulation 1.441-2 [1] (IRS Publication 538).,[2] as well as the 
International Financial Reporting Standards.

'''
for file in files_to_process:
    df = pd.read_excel(file)
    df.fillna(0, inplace=True)
    df['fiscal_yr_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR')
    df['month'] = df.task_date.dt.month
    df['Month_txt'] = df['month'].apply(lambda x: calendar.month_abbr[x]) # 0 is empty string,a number converts to 
                                                                        # that month's number Aug = 8, x gives whatever number the month is.
    df['year'] = df.task_date.dt.year
    df['day'] = df.task_date.dt.day
    df['fiscal_qtr'] = pd.PeriodIndex([date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.FY5253Quarter(normalize=True, weekday=4,startingMonth=4, qtr_with_extra_week=1, variation='last') for date in df.task_date], freq='Q-APR').quarter
    df.loc[((df.task_date.dt.month >= 5) | ((df.task_date.dt.month == 4) & (df['fiscal_qtr'] == 1))), 'fiscal_yr'] = df.task_date.dt.year + 1
    df.loc[((df.task_date.dt.month <= 4) & (df['fiscal_qtr'] != 1)), 'fiscal_yr'] = df.task_date.dt.year
    df['total_expenses'] = df['costs'] + df['3rd_prty']
    df['total_spend'] = df['fees'] + df['costs'] + df['3rd_prty']
    df.to_excel(file, index=False)
    with pd.ExcelWriter('output.xlsx', datetime_format='YYYY-MM-DD') as writer:
        df.to_excel(writer, sheet_name='Sheet_name_1')

'''
adding timestamp to files_to_process that are now processed
'''
timestmp_prefix = ('REVISED'
                   + str(pd.Timestamp.today().year)
                   + str(pd.Timestamp.today().month)
                   + str(pd.Timestamp.today().day)
                   + str(pd.Timestamp.today().hour) 
                   + str(pd.Timestamp.today().minute))

'''
i read the following list comprehension as taking file f and formatting the ouput name 
to be the prefix followed by the original file name
for all files in the directory meeting the filename criteria in the glob function above
'''
[os.rename(f, "{}{}".format(timestmp_prefix, f)) for f in files_to_process]

'''
move the forecasts that were the new to be forecasts 
that are now processed and have a timestamp prefix added to their name 
to a processed file folder "submitted_forecast_processed"
so we don't process the forecasts twice
'''
# files_to_move = sorted(glob('*CAH_forecast*.xlsx'))
# # files_to_move = file_to_process

# destination = "forecasts_under_construction/"
# for files in files_to_move:
#     if files.endswith(".xlsx"):
#         shutil.move(files, destination)
df

['2020.01.13_Avanos_CAH_KEY_STRATEGY_DOC_(182850940)_(1).xlsx']


,category,task,in_scope,task_date,fees,costs,3rd_prty,assumptions_notes,needs_discussion,firm_name,...,fiscal_qtr,fiscal_yr_qtr,total_expenses,total_spend,timestamp,frcst_status,IP_budget_template_ phase,IP_budget_template_ row_num,Jen,Month_txt
0,experts,work with expert,Y,2000-01-01,0,0,0,"Retain expert, collect/analyze data; damages e...",0,0,...,3,2000Q3,0,0,0,0,Post-Markman,38-41,"Damages work (expert, document collection/review)",Jan
1,experts,work with expert,Y,2021-01-15,0,0,35714,Opening Expert reports; work with all experts,0,0,...,3,2021Q3,35714,35714,0,0,Post-Markman,38-41,Expert Report Drafting,Jan
2,experts,work with expert,Y,2000-01-01,0,0,0,Opening Expert reports; work with all experts,0,0,...,3,2000Q3,0,0,0,0,Post-Markman,38-41,Expert Report Drafting,Jan
3,IPR_prepare_IPR,work with expert,Y,2000-01-01,0,0,0,chaz,0,0,...,3,2000Q3,0,0,0,0,0,0,jen?,Jan
4,trial_preparation,Witness List,Y,2000-01-01,0,0,0,chaz,0,0,...,3,2000Q3,0,0,0,0,Trial Preparation and Trial,52,jen?,Jan
5,strategy,weekly calls and standing agenda,Y,2000-01-01,0,0,0,chaz,0,0,...,3,2000Q3,0,0,0,0,0,0,jen?,Jan
6,trial_preparation,Voir Dire Questions,Y,2000-01-01,0,0,0,chaz,0,0,...,3,2000Q3,0,0,0,0,Trial Preparation and Trial,52,jen?,Jan
7,trial_preparation,Verdict Form,Y,2000-01-01,0,0,0,chaz,0,0,...,3,2000Q3,0,0,0,0,Trial Preparation and Trial,52,jen?,Jan
8,scheduling order,Trial ready,Y,2000-01-01,0,0,0,chaz,0,0,...,3,2000Q3,0,0,0,0,0,0,jen?,Jan
9,trial_preparation,Trial Preparation,Y,2000-01-01,0,0,0,chaz,0,0,...,3,2000Q3,0,0,0,0,Trial Preparation and Trial,52,jen?,Jan


In [ ]:
print(df.columns)
ptable = pd.pivot_table(df, values=['fees', 'costs', '3rd_prty'], index=['fiscal_yr', 'fiscal_qtr'], aggfunc=np.sum, fill_value=0)
ptable

In [ ]:
ptable = pd.pivot_table(df, values=['fees', 'total_expenses'], index=['fiscal_yr', 'fiscal_qtr'], aggfunc=np.sum, fill_value=0)
ptable

In [ ]:
ptable = pd.pivot_table(df, values=['total_spend'], index=['fiscal_yr', 'fiscal_qtr'], aggfunc=np.sum, fill_value=0)
ptable

In [ ]:
print(os.getcwd())
os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
os.chdir(r'J:\LEGAL\share1\IP_forecasting_test_env\accruals')

print(os.getcwd())
# os.chdir('J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20\Q4\Apr')
print(os.getcwd())
print(os.listdir())
colNames = ["MDT Matter #", "Total Unbilled"] #, "Case Name/ Description"
#df = pd.read_excel('Robins Kaplan April Accrual (Spine).xlsx', usecols=colNames, sheet_name='Unbilled', index=0, header=4)
# df.head()
files_to_process = sorted(glob('*.xlsx'))
print(files_to_process)

for file in files_to_process:
    df = pd.read_excel(file, usecols=colNames, sheet_name='Unbilled', index=0, header=4)
    # print(df.head())
    #df2 = pd.concat([df2, df], sort=True)
    print(df.head())
    


print('chaz')




# os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
# print(os.getcwd())
# os.listdir()

In [ ]:
df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'])

df2 = pd.DataFrame([['c', 33], ['d', 4]],
                   columns=['letter', 'number'])

df3 = pd.concat([df1, df2])
df1
df3


In [6]:
'''this accrual gathering program gets the accruals from Daniel accrual file
And this iterates through the entire fiscal year
'''
print(os.getcwd())
os.chdir('J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20')
rootdir = 'J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20'
for subdir, dirs, files in os.walk(rootdir):
    print(subdir, 'chaz')
    print(dirs, 'chaz_1')
    print(files, 'chaz_2')


J:\LEGAL\share1\IP_forecasting_test_env
J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20 chaz
['Q1', 'Q3', 'Q2', 'Q4'] chaz_1
[] chaz_2
J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20\Q1 chaz
['Originals', 'DC Edits'] chaz_1
[] chaz_2
J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20\Q1\Originals chaz
['May', 'June', 'July'] chaz_1
[] chaz_2
J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20\Q1\Originals\May chaz
[] chaz_1
['Ropes Gray May Accrual.xlsx', 'Pepper May Accrual.xlsx', 'Jones Day May Accrual .xlsx', 'SHB May Accrual.xlsx', 'Greenberg Traurig May Accrual.xlsx', 'Maslon May Accruals (Diabetes).xlsx', 'Stikeman May Accrual.xlsx', 'Fredrikson Byron May Accruals.xlsx', 'WilmerHale May Accruals.xlsx', 'Winston Strawn May Accrual.xlsx', 'Williams Connolly May Accrual.xlsx', 'Robins Kaplan May Accruals.xlsx', 'Fredrikson Byron May Accruals (Dubo).xlsx', 'EY May Acrrual .xlsx', 'MRC May Accrual.xlsx', 'DLA May Accruals.xlsx'] chaz_2
J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20\Q1\Originals\June chaz
[] chaz_1


J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20\Q4 chaz
['Apr', 'Feb', 'Mar'] chaz_1
[] chaz_2
J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20\Q4\Apr chaz
[] chaz_1
['SHB Apr Accrual (MITG).xlsx', 'EY Apr Accrual (MITG, Spine).xlsx', 'Greenberg Traurig Apr Accrual (NV).xlsx', 'Ropes Gray Apr Accrual (MITG, NV).xlsx', 'DLA Apr Accrual (MITG, Diab).xlsx', 'Littler Apr Accrual (MITG).xlsx', 'Maslon Apr Accrual (Diab).xlsx', 'Jones Day Apr Accrual (MITG, Spine, NV).xlsx', 'Stikeman Elliott Apr Accrual (Diab, MITG).xlsx', 'WilmerHale Apr Accrual (MITG, Spine).xlsx', 'MRC Apr Accual (NV, Diab).xlsx', 'Robins Kaplan April Accrual (Spine).xlsx'] chaz_2
J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20\Q4\Feb chaz
[] chaz_1
['Winston Strawn Feb Accrual (None).xlsx', 'SHB Feb Accrual (MITG).xlsx', 'Jones Day Feb Accrual (MITG, NV).xlsx', 'Maslon Feb (Diab) .xlsx', 'EY Feb Accrual .xlsx', 'WilmerHale Feb Accrual (Spine, MITG).xlsx', 'DLA Piper Feb Accrual (Diab, MITG).xlsx', 'Littler Feb Accrual (Diab, MITG).xlsx

In [17]:
'''this is going to be a way to iterate through a file directory and print the files (this is for TYRX matter today)
\\mspm1bnas50s.ent.core.medtronic.com\TYRX
'''
print(os.getcwd())
os.chdir('/HANSOC10-L7/mspm1bnas50s.ent.core.medtronic.com/TYRX')
print(os.getcwd())
# rootdir = 'J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20'
# for subdir, dirs, files in os.walk(rootdir):
#     print(subdir, 'chaz')
#     print(dirs, 'chaz_1')
#     print(files, 'chaz_2')

J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/HANSOC10-L7/mspm1bnas50s.ent.core.medtronic.com/TYRX'

In [ ]:
'''this accrual gathering program gets the accruals from Daniel accrual file
And this iterates through the entire fiscal year
'''
print(os.getcwd())
os.chdir('J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20')
rootdir = 'J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20'

'''
set the column names I want to use our of the accrual excel file from firms
'''
colNames = ["MDT Matter #", "Total Unbilled", "Case Name/ Description"]

'''
set up an empty dateframe with the column names I set to capture the combined accruals
'''
result = pd.DataFrame(columns=colNames)
print(result)

for subdir, dirs, files in os.walk(rootdir):
    print(subdir, 'chaz')
    print(dirs, 'chaz_1')
    print(files, 'chaz_2')

    for file in files:
        try:
            df = pd.read_excel(file, usecols=colNames, sheet_name='Unbilled', index=0, header=4)
            df['accrual_file'] = file # this adds each file name so I can go back easily to look at the accrual 
            print('we in try')
        except:
            print('The following file lacks sheet_name=Unbilled: ', file)
            pass
        result = result.append(df, sort=False) # each new file added to the combined file
        
        #print(df.head())
    # df2 = pd.concat([df2, df], sort=True)
    # print(df.head())
    # result = df.append([result, df])

os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
result.to_excel('accrual_combo2.xlsx', index=False)
print(os.getcwd())
print('done')



In [ ]:
os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
'''
I set up a folder to iterate through for accruals
'''

os.chdir(r'J:\LEGAL\share1\IP_forecasting_test_env\accruals')

'''print current directory making sure I know where I am.
'''
print(os.getcwd())

'''create a list of file names with glob
'''
files_to_process = sorted(glob('*.xlsx'))

'''what is the list of files
'''
print(files_to_process)

'''set the column names I want to use our of the accrual excel file from firms
'''
colNames = ["MDT Matter #", "Total Unbilled", "Case Name/ Description"]

'''set up an empty dateframe with the column names I set to capture the combined accruals
'''
result = pd.DataFrame(columns=colNames)

'''iterate through the files and make a combined database of accruals
'''
for file in files_to_process:
    try:
        df = pd.read_excel(file, usecols=colNames, sheet_name='Unbilled', index=0, header=4)
        df['accrual_file'] = file # this adds each file name so I can go back easily to look at the accrual 
    except:
        print('The following file lacks sheet_name=Unbilled: ', file) # some accrual files aren't set up right
        pass
    result = result.append(df, sort=False) # each new file added to the combined file

'''save the combined file to an excel document for use later
'''
result.to_excel('accrual_combobb.xlsx', index=False)

# working as of 4/30/2020


In [ ]:
print(os.getcwd())
os.chdir('J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20')
rootdir = 'J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY20'

'''set the column names I want to use our of the accrual excel file from firms
'''
colNames = ["MDT Matter #", "Total Unbilled", "Case Name/ Description"]

'''set up an empty dateframe with the column names I set to capture the combined accruals
'''
result = pd.DataFrame(columns=colNames)

'''iterate through the files and make a combined database of accruals
'''


for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(file, 'this is file')
        print(os.path.join(subdir, file))
        file_to_eval = os.path.join(subdir, file)
        print('file-to-eval:', file_to_eval)
        try:
            df = pd.read_excel(file_to_eval, usecols=colNames, sheet_name='Unbilled', index=0, header=4)
            df['accrual_file'] = file # this adds each file name so I can go back easily to look at the accrual
            print(df.head())
            result = result.append(df, sort=False) # each new file added to the combined file
        except:
            print('The following file lacks sheet_name=Unbilled: ', file) # some accrual files aren't set up right
            pass
 
os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
result.to_excel('accrual_combo2.xlsx', index=False)
print(os.getcwd())
print('done')
# this is working



In [ ]:
os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
os.chdir(r'J:\LEGAL\share1\IP_forecasting_test_env\accruals')
print(os.getcwd())
file_list = os.listdir()

colNames = ["MDT Matter #", "Total Unbilled", "Case Name/ Description"]
result = pd.DataFrame(columns=colNames)
rootdir = r'J:\LEGAL\share1\IP_forecasting_test_env\accruals'
for subdir, dirs, files in os.walk(rootdir):
    print(subdir, 'chaz')
    print(dirs, 'chaz_1')
    print(files, 'chaz_2')

#print(os.path.join(subdir, file))
file_to_eval = os.path.join(rootdir, file)
print('this is file to eval:', file_to_eval)
# print('file-to-eval:', file_to_eval)
# try:
df = pd.read_excel(file_to_eval, usecols=colNames, sheet_name='Unbilled', index=0, header=4)
df['accrual_file'] = file # this adds each file name so I can go back easily to look at the accrual
#             print(df.head())
result = result.append(df, sort=False) # each new file added to the combined file
print(result.head())
#except:
#             print('The following file lacks sheet_name=Unbilled: ', file) # some accrual files aren't set up right
#             pass
df 
# working as of evening of 4/29/2020 but as you see no iteration

In [ ]:
os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
os.chdir(r'J:\LEGAL\share1\IP_forecasting_test_env\accruals')
# print(os.getcwd())
file_list = os.listdir()


colNames = ["MDT Matter #", "Total Unbilled", "Case Name/ Description"]
result = pd.DataFrame(columns=colNames)

for file in file_list:
    # print(file)
    df = pd.read_excel(file_to_eval, usecols=colNames, sheet_name='Unbilled', index=0, header=4)
    df['accrual_file'] = file # this adds each file name so I can go back easily to look at the accrual
    #print(df.head())
    result = result.append(df, sort=False) # each new file added to the combined file
result
# working as of evening og 4/29/2020


In [ ]:
#used this 6/29/2020
print(os.getcwd())
os.chdir('J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY21')
rootdir = 'J:\LEGAL\RIMS\Daniel\Monthly Accruals\FY21'

'''set the column names I want to use our of the accrual excel file from firms
'''
colNames = ["MDT Matter #", "Total Unbilled", "Case Name/ Description"]

'''set up an empty dateframe with the column names I set to capture the combined accruals
'''
result = pd.DataFrame(columns=colNames)

'''iterate through the files and make a combined database of accruals
'''


for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(file, 'this is file')
        print(os.path.join(subdir, file))
        file_to_eval = os.path.join(subdir, file)
        print('file-to-eval:', file_to_eval)
        try:
            df = pd.read_excel(file_to_eval, usecols=colNames, sheet_name='Unbilled', index=0, header=4)
            df['accrual_file'] = file # this adds each file name so I can go back easily to look at the accrual
            print(df.head())
            result = result.append(df, sort=False) # each new file added to the combined file
        except:
            print('The following file lacks sheet_name=Unbilled: ', file) # some accrual files aren't set up right
            pass
 
os.chdir('J:\LEGAL\share1\IP_forecasting_test_env')
result.to_excel('accrual_combo23.xlsx', index=False)
print(os.getcwd())
print('done')
# this is working
